# Vigenere cipher cryptoanalysis


This notebook studies the cryptoanalysis of the [Vigenere][wiki] cipher, which is polyalphabetic, namely two occurrences  $a_{1}, a_{2}$ of character $a$ belonging to the plaintext are associated with occurrences $c_{1}, c_{2}$ in the ciphertext, such that $c_{1}\neq c_{2}$ with very high probability.

Our implementation closely follows a class lecture given by Prof. [Orazio Puglisi][puglisi] within Cryptography course at the University of Florence. In order to fully understand the last part, where the concept of *mutual coincidence index* is crucial, we rest on the explanation at pag. 20 of [this notes][notes].

[wiki]:https://en.wikipedia.org/wiki/Vigen%C3%A8re_cipher
[puglisi]:http://web.math.unifi.it/users/puglisi/
[notes]:http://iml.univ-mrs.fr/~ritzenth/cours/crypto.general.pdf

In [2]:
import itertools
from itertools import *
from copy import copy, deepcopy
from heapq import *
from random import *

from sympy import *

init_printing()

---

Let $\mathcal{A}$ be our alphabet composed of standard English characters plus the `space` character. It will be sufficient to encode simple and (not so quite) short messages. To define it as a group $\frac{\mathbb{Z}}{n\mathbb{Z}}$ in the Python language we use a `dict` object, which can be reversed because it is a bijection.

In [3]:
def make_alphabet_entry(i):
    alpha = i + ord('a')
    return chr(alpha),i

A = dict(map(make_alphabet_entry, range(26)))
A.update({' ':26})
inverse_A = {v:k for k,v in A.items()}

A

{' ': 26,
 'a': 0,
 'b': 1,
 'c': 2,
 'd': 3,
 'e': 4,
 'f': 5,
 'g': 6,
 'h': 7,
 'i': 8,
 'j': 9,
 'k': 10,
 'l': 11,
 'm': 12,
 'n': 13,
 'o': 14,
 'p': 15,
 'q': 16,
 'r': 17,
 's': 18,
 't': 19,
 'u': 20,
 'v': 21,
 'w': 22,
 'x': 23,
 'y': 24,
 'z': 25}

We define a function `encode` that consumes a string and produces a list of integer in our field. In parallel, function `decode` goes backwards, it consumes a list of integers and return a string. Such functions are useful in order to use the cipher and analyze it using `str` objects instead of the coded version using lists of integers.

In [4]:
def encode(s, alphabet=A): return list(map(lambda i: alphabet[i], s))

def decode(e, inverse_alphabet=inverse_A): return "".join(map(lambda i: inverse_alphabet[i], e))

The following plaintext is a prose taken from [here]; before using it we have to swipe out punctuation marks:

[here]:http://www.bartleby.com/209/2.html

In [5]:
with open('plain_text.txt', 'r') as f:
    plain_text = f.read().lower()

remove_chars = [',', '.', ';', ':', '-', '(', ')', "'", '"']
for rc in remove_chars: 
    plain_text = plain_text.replace(rc, '')
plain_text = plain_text.replace('\n', ' ')

plain_text = "".join(filter(lambda c: not c.isdigit(), plain_text))
encoded_plain_text = encode(plain_text)

plain_text, len(plain_text)

('the attraction of medieval literature comes perhaps more strongly from some other countries than from england in france and provence in germany and iceland there were literary adventurers more daring and achievements more distinguished it was not in england that the most wonderful things were produced there is nothing in old english that takes hold of the mind with that masterful and subduing power which still belongs to the lyrical stanzas of the troubadours and minnesingers to welsh romance or to the epic prose of the iceland histories   the norman conquest degraded the english language from its literary rank and brought in a new language for the politer literature it did not destroy in one sense it did not absolutely interrupt english literature but it took away the english literary standard and threw the country back into the condition of italy before dantean anarchy of dialects when a new literary language was established in the time of chaucer the middle ages were nearly over a

With the following assert we ensure that function `decode` is the inverse of function `encode`:

In [6]:
assert decode(encode(plain_text)) == plain_text

Let $\textbf{k}=(k_{0},\ldots,k_{m-1}) \in \left(\frac{\mathbb{Z}}{n\mathbb{Z}}\right)^{m}$ be a *key* of length $m\in\mathbb{N}$. In order to have a meaningful analysis we build a function `generate_random_key` which generate a random key of random length, keeping it safe...we will uncover it only at the end to check our work:

In [29]:
def generate_random_key(given_key=None, required_length=None, max_length=None, alphabet=A):
    
    if given_key is not None: return given_key
    
    if required_length is None and max_length is None: max_length = len(alphabet)
    
    # the minimum length of the key is 3 to build interesting cases
    length = required_length if required_length else randint(3, max_length)
    key = [0] * length
    # -1 in the following max limit because it is inclusive in the sense of `randint`.
    for i in range(length): key[i] = randint(0, len(alphabet)-1)
    return key

#key = encode("ericsmullyan")
secret_key = generate_random_key(required_length=17)

In [30]:
len(secret_key), decode(secret_key)

(17, 'cdkgznhzlvdfnzskg')

Now we are in the position to define `encrypt` and `decrypt` functions, both of them consumes an *encoded* message, namely a list of integers, and a key. The encryption work by repeating the key as long as necessary to match the same length of the input $\textbf{x}=(x_{0},\ldots,x_{h})$, where $h > m-1$ otherwise the cipher is a *OneTimePad* which is unbreakable, 
$$(\underbrace{x_{0},\ldots,x_{m-1}}_{k_{0},\ldots,k_{m-1}}
\underbrace{x_{m},\ldots,x_{m-1}}_{k_{0},\ldots,k_{m-1}}
\ldots
\underbrace{x_{lm},\ldots,x_{h}}_{k_{0},\ldots,k_{h \mod m}}) = 
(y_{0},\ldots,y_{h}) = \textbf{y}$$
truncating the very right block to match plaintext suffix. At last, the ciphertext $\textbf{y}$ is obtained by addition modulo $n$ of corresponding symbols, where $n$ is the length of the alphabet. Decryption follows the same scheme, using modular subtraction.

In [9]:
def encrypt(message, key, alphabet=A):
    n = len(alphabet)
    return [(p+v)%n for p,v in zip(message, cycle(key))]

def decrypt(cipher, key, alphabet=A):
    n = len(alphabet)
    return [(c-v)%n for c,v in zip(cipher, cycle(key))]

the following is the *ciphertext* produced by the application of function `encrypt` to the plaintext:

In [31]:
cipher_text = encrypt(encoded_plain_text, secret_key)
decode(cipher_text)

'vkofzf plxwnalrylbpojgrbzwuonfcikzwuofaatccusjdfszybpyxcmzrbiqlywrpxqpjymzlyznkjdyuy pwaocegrsvqespfwfgqqrz kythckdzemkbdxjybymfzqhry xfihasz eylhgevawvgpgjzfryckqhwrycszgukxwmhbfzqygpwaybpyxcmkzbcqlmzenfcfrochlkphwxmkfakbgsyrvueecvmrbrszbzkyy vrkcqerlyvgpgjzfn ydbhezmjcfyrxjcdmswuwmvlybfyhakybymoofjqykrkthjoqmumdblsty xfqonfc njtmkeffscfvdukqmomwyctsykrkbpstbmcgdbcyuzkjscvckpsajkvqimqlljwlxmybvuplcaugurfuwsrjmicwiqleykyfvkofjkygnvoeersxecvjudm fpuwwastkjqxayynubkgls cjstihayyfvygzoxuyiyscqmkyayydicyucrovkfjvpazckiieffwjoehvglqgftmwtdgwbfbccncmumbgdsmafxwwhbzyqlebvgjqykrkbhxmjvzfkfdstssqkbiaukmprcuonfcikx caglxgzyycgdmlqnvcstynglpqcqnlydgihjlmdgrszcuaj cktcvorryzdoujmgkjjkgjtmfgbpmwwawrstbrxkyellczcnfyvsjbqyzyniqzfxyrjpjopwoxpgwrkzqlygjrfnlckpn sbzcggrrszbwyuimhulscyucrotiosyfmsgdzufdwrbzcqngpqgzyycyupwffvkofaaaldlaeozuufkqcuyfockxrsqgksupcylyv zwscgrdfakbgktrrhlkvqfdazhfqijjgnscnnveifwxfccxkumsgdzufdwrvgpjdgergulmcjerslrkvrkbmplknkjmr wkbrpfauhsnzueffwjskgnrcmhepmcarpwjtgdarwmvtplcf 

Assuming to know the `secret_key`, we ensure that function `decrypt` is the inverse of function `encrypt`:

In [32]:
assert decode(decrypt(cipher_text, secret_key)) == plain_text

Let $I_{c}$ be the *coincidence index* of a sequence $\alpha$, over an alphabet $A$, defined as 
$$ I_{c}(\alpha) = \sum_{i=0}^{n}{\frac{f_{a_{i}}(f_{a_{i}}-1)}{m(m-1)}} $$ where $n$ is the length of the alphabet, $m$ is the length of $\alpha$ and $f_{a_{i}}$ is the frequency of symbol $a_{i}\in A$, namely the number of occurrences of $a_{i}$ in $\alpha$. In other words, $I_{c}$ is the probability to sample two occurrences of the same symbol $a$, forall $a\in A$, from the sequence $\alpha$. 

Index $I_{c}$ is invariant to shifting by the same constant $v$. Let $\alpha,\beta$ be two sequences of integers of length $l$ such that $\beta_{i} \equiv_{n} \alpha_{i} + v$, forall $i\in\lbrace0,\ldots,l\rbrace$; moreover, let $q_{\gamma_{i}}^{(\gamma)}$ be the probability to sample *two* occurrences of $\gamma_{i}$ from a sequence $\gamma$, we can state the relations
$$
    I_{c}(\beta) = \sum_{i\in\frac{\mathbb{Z}}{n\mathbb{Z}}}{q_{\beta_{i}}^{(\beta)}}
                = \sum_{i\in\frac{\mathbb{Z}}{n\mathbb{Z}}}{q_{\beta_{i}-v}^{(\alpha)}}
                = \sum_{i\in\frac{\mathbb{Z}}{n\mathbb{Z}}}{q_{\alpha_{i}}^{(\alpha)}}
                = I_{c}(\alpha)
$$
proving the invariance of $I_{c}$ when a sequence is produced by another one shifted by a constant $v\in\frac{\mathbb{Z}}{n\mathbb{Z}}$.

In [33]:
def frequencies(lst, alphabet=A, inverse_alphabet=inverse_A):
    """ Returns a `dict` counting occcurrences of each object of the alphabet within the iterable.
    
    `frequencies` consumes an iterable $lst$ and an alphabet $A$, 
    produces a dictionary of entries $(k,v)$ where $k$ is a character 
    in $A$ and $v$ is the number of occurrences of $k$ in $lst$"""
    
    freqs = {k:0 for k in alphabet.keys()}
    for c in lst: freqs[inverse_alphabet[c]] += 1
    return freqs

def length_from_frequencies(freqs):
    return sum(v for v in freqs.values())

def coincidence_index(freqs, alphabet=A):
    denom = length_from_frequencies(freqs)
    if denom in range(2): return None
    return sum(v*(v-1) for v in freqs.values())/(denom*(denom-1))

In [34]:
plaintext_frequencies = frequencies(encoded_plain_text)
plaintext_frequencies

{' ': 335,
 'a': 115,
 'b': 19,
 'c': 40,
 'd': 67,
 'e': 215,
 'f': 34,
 'g': 39,
 'h': 81,
 'i': 113,
 'j': 0,
 'k': 9,
 'l': 57,
 'm': 29,
 'n': 140,
 'o': 120,
 'p': 21,
 'q': 1,
 'r': 125,
 's': 93,
 't': 162,
 'u': 35,
 'v': 14,
 'w': 29,
 'x': 0,
 'y': 26,
 'z': 1}

In [35]:
print("coincidence index of *plaintext*: {}\ncoincidence index of *ciphertext*: {}".format(
        coincidence_index(plaintext_frequencies), coincidence_index(frequencies(cipher_text))))

coincidence index of *plaintext*: 0.07878506600660067
coincidence index of *ciphertext*: 0.0411873588674657


In [36]:
def spread(message, block_length):
    return [message[i:i+block_length] for i in range(0, len(message), block_length)]
    
def col(spreaded, c, join=False, joiner=lambda c: ''.join(decode(c))):
    column = [lst[c] if c < len(lst) else None for lst in spreaded]
    ready = list(filter(lambda i: i is not None, column))
    return joiner(ready) if join else ready

def decode_spreaded(spreaded, join_as_str=False):
    decoded_spread = list(map(decode, spreaded))
    return '\n'.join(decoded_spread) if join_as_str else decoded_spread

def analyze(cipher_text):
    res = {}
    # we discard the case where the key length equals the
    # length of the cipher text, since it is the case of 
    # OneTimePad cipher, which is unbreakable!
    for d in range(2, len(cipher_text)):
        spreaded = spread(cipher_text, d)
        res[d] = []
        for c in range(d):
            ci = coincidence_index(frequencies(col(spreaded, c)))
            if ci: res[d].append(ci)
                
    return res

def guess_key_length(analysis, threshold=0.06):
    candidates = {}
    for k,v in analysis.items():
        cs = list(filter(lambda i: i > threshold, v))
        if cs and len(cs) > ceiling(k/2): candidates[k] = cs
    return candidates

In [37]:
analysis = analyze(cipher_text)

In [38]:
probing_key_length = 17
guess_key_length(analysis)[probing_key_length]

In [39]:
decode_spreaded(spread(cipher_text, probing_key_length))

['vkofzf plxwnalryl',
 'bpojgrbzwuonfcikz',
 'wuofaatccusjdfszy',
 'bpyxcmzrbiqlywrpx',
 'qpjymzlyznkjdyuy ',
 'pwaocegrsvqespfwf',
 'gqqrz kythckdzemk',
 'bdxjybymfzqhry xf',
 'ihasz eylhgevawvg',
 'pgjzfryckqhwrycsz',
 'gukxwmhbfzqygpway',
 'bpyxcmkzbcqlmzenf',
 'cfrochlkphwxmkfak',
 'bgsyrvueecvmrbrsz',
 'bzkyy vrkcqerlyvg',
 'pgjzfn ydbhezmjcf',
 'yrxjcdmswuwmvlybf',
 'yhakybymoofjqykrk',
 'thjoqmumdblsty xf',
 'qonfc njtmkeffscf',
 'vdukqmomwyctsykrk',
 'bpstbmcgdbcyuzkjs',
 'cvckpsajkvqimqllj',
 'wlxmybvuplcaugurf',
 'uwsrjmicwiqleykyf',
 'vkofjkygnvoeersxe',
 'cvjudm fpuwwastkj',
 'qxayynubkgls cjst',
 'ihayyfvygzoxuyiys',
 'cqmkyayydicyucrov',
 'kfjvpazckiieffwjo',
 'ehvglqgftmwtdgwbf',
 'bccncmumbgdsmafxw',
 'whbzyqlebvgjqykrk',
 'bhxmjvzfkfdstssqk',
 'biaukmprcuonfcikx',
 ' caglxgzyycgdmlqn',
 'vcstynglpqcqnlydg',
 'ihjlmdgrszcuaj ck',
 'tcvorryzdoujmgkjj',
 'kgjtmfgbpmwwawrst',
 'brxkyellczcnfyvsj',
 'bqyzyniqzfxyrjpjo',
 'pwoxpgwrkzqlygjrf',
 'nlckpn sbzcggrrsz',
 'bwyuimhu

In [40]:
def mutual_coincidence_index(fst_freqs, snd_freqs, offset=0, 
                             alphabet=A, inverse_alphabet=inverse_A):
    fst_len = length_from_frequencies(fst_freqs)
    snd_len = length_from_frequencies(snd_freqs)
    n = len(alphabet)
    return sum(fst_freqs[k] * snd_freqs[inverse_alphabet[(v+offset) % n]] 
               for k,v in alphabet.items())/(fst_len * snd_len)

In [41]:
def build_offsets_eqs(cipher_text, key_length, indexed_sym, threshold=.06, alphabet=A):
    
    n = len(alphabet)
    eqs = {c:{} for c in range(key_length)}
    spreaded = spread(cipher_text, key_length)
    for c,a in itertools.product(range(key_length), repeat=2):
    
        if a == c: continue
        
        eqs[c][a]=[]
        
        for g in range(1,n):
            column_freqs = frequencies(col(spreaded, c))
            another_freqs = frequencies(col(spreaded, a))
            mci = mutual_coincidence_index(column_freqs, another_freqs, g)
            if mci > threshold: 
                eqs[c][a].append(tuple([Eq(indexed_sym[a]-indexed_sym[c],g,evaluate=True), mci]))
    
    return eqs

In [42]:
k_sym=IndexedBase('k')

eqs_dict = build_offsets_eqs(cipher_text, key_length=probing_key_length, indexed_sym=k_sym)

In [43]:
eqs_dict[0]

In [44]:
def clean_mci_info(eqs_dict):
    
    res = {}
    for c, eq_dict in eqs_dict.items():
        eqs_list = []
        for a, eqs in eq_dict.items():
            if not eqs: 
                eqs_list = []
                break
            eqs_list.append([[a] + list(eq_pair) for eq_pair in eqs])
        res[c]=list(itertools.product(*eqs_list))
    
    return res

In [45]:
eqs_dict_pure = clean_mci_info(eqs_dict)
eqs_dict_pure

In [51]:
def candidate_keys(eqs_dict, indexed_sym, alphabet=A):
    key_length = len(eqs_dict)
    n = len(alphabet)
    candidates=set()
    for c, eqs_tuples in eqs_dict.items():
        for d in range(len(alphabet)):
            for eq_tuple in eqs_tuples:
                key = [indexed_sym[i] for i in range(key_length)]
                key[c] = d
                score = 1
                for a, eq, mci in eq_tuple:
                    score *= mci
                    subs_eq = eq.subs(indexed_sym[c],d)
                    key[a] = solve(subs_eq, indexed_sym[a])[0]
                    key[a] = key[a] % n

                key_is_fully_instantiated = True
                for k in key:
                    if isinstance(k, Indexed): key_is_fully_instantiated = False 
                
                if key_is_fully_instantiated:
                    elem = (1-score, key)
                    #heappush(candidates, elem)
                    candidates.add(tuple(key))
            
    return candidates

In [52]:
possible_keys = candidate_keys(eqs_dict_pure, k_sym)

In [57]:
def attempt_keys(candidate_keys, cipher_text, threshold=.06):
    sols = set()
    for key in candidate_keys:
        decrypted = decrypt(cipher_text, key)
        ci = coincidence_index(frequencies(decrypted))
        if ci > threshold: sols.add(decode(key))#sols.add((ci, decode(key)))
    return sols

In [58]:
sols = attempt_keys(possible_keys, cipher_text)

In [62]:
len(sols), sols

(324,
 {' ahdwkewisacbwphd',
  ' ahdwkewisackwphd',
  ' ahdwkewisackwphq',
  ' ahdwkewixacbwphd',
  ' ahdwkewixackwphd',
  ' ahdwkewnsacbwphd',
  ' ahdwkewnsackwphd',
  ' ahdwkewnxacbwphd',
  ' ahdwkewnxackwphd',
  ' fmiapjanxfhpaumi',
  ' nhdwkewisackwphd',
  ' nhdwkewisackwphq',
  'abiexlfxjtbdcxqie',
  'abiexlfxjtbdlxqie',
  'abiexlfxjtbdlxqir',
  'abiexlfxjybdcxqie',
  'abiexlfxjybdlxqie',
  'abiexlfxotbdcxqie',
  'abiexlfxotbdlxqie',
  'abiexlfxoybdcxqie',
  'abiexlfxoybdlxqie',
  'agnjbqkboygiqbvnj',
  'aoiexlfxjtbdlxqie',
  'aoiexlfxjtbdlxqir',
  'bcjfymgykucedyrjf',
  'bcjfymgykucemyrjf',
  'bcjfymgykucemyrjs',
  'bcjfymgykzcedyrjf',
  'bcjfymgykzcemyrjf',
  'bcjfymgypucedyrjf',
  'bcjfymgypucemyrjf',
  'bcjfymgypzcedyrjf',
  'bcjfymgypzcemyrjf',
  'bhokcrlcpzhjrcwok',
  'bpjfymgykucemyrjf',
  'bpjfymgykucemyrjs',
  'cdkgznhzl dfezskg',
  'cdkgznhzl dfnzskg',
  'cdkgznhzlvdfezskg',
  'cdkgznhzlvdfnzskg',
  'cdkgznhzlvdfnzskt',
  'cdkgznhzq dfezskg',
  'cdkgznhzq dfnzskg',
  'cd

In [63]:
k1 = encode('abiexlfxjtbdcxqie')
d = decrypt(cipher_text, k1)
assert encrypt(d, k1) == cipher_text
decode(d), coincidence_index(frequencies(d))

('vjgbcvvtcevkzpbqhbogfkgxcnbnkdgtcvwtgbeqogubrgbjcruboqtgbuvtqpiw bhtqobuqogbqvjgbbeqwpvtkgubvjcpbqtqobgpincpfbkpbhbcpegbcpfbrtqxgpepbkpbigtocp bcpfbtegncpfbvjgtgbygtpbnkvgtct bcfxgpvetgtuboqtgbfctkpikcpfbcejkgxgogpvukoqtgbfkuvkpiwkujpfbkvbycubpqvbkpbppincpfbvjcvbvjgbxquvbyqpfgthwnbvjtpiubygtgbrtqfwegobvjgtgbkubpqvjkprbkpbqnfbgpinkujbdjcvbvcmgubjqnfbqqbvjgbokpfbykvjbvscvbocuvgthwnbcpfkuwdfwkpibrqygtbyskejbuvknnbdgnqpicbvqbvjgbn tkecnbcvcpacubqhbvjgbvtzwdcfqwtubcpfbokpygukpigtubvqbygnusbtqocpegbqtbvqbvsgbgrkebrtqugbqhbdjgbkegncpfbjkuvqbkgubbbvjgbpqtocpkeqpswguvbfgitcfgobvjgbgpinkujbncprwcigbhtqobkvubnkdgtct btcpmbcpfbdbqwijvbkpbcbpgybnlpiwcigbhqtbvjgbrznkvgtbnkvgtcvwtgkkvbfkfbpqvbfguvtz bkpbqpgbugpugbkdbfkfbpqvbcduqnwvpn bkpvgttwrvbgpiwkujbnkvgtcvwtgbdevbkvbvqqmbcyc bvsgbgpinkujbnkvgtcb buvcpfctfbcpfbvstgybvjgbeqwpvt bmcembkpvqbvjgbeqpokvkqpbqhbkvcn bdphqtgbfcpvgcpbcpcbej bqhbfkcngevubgjgpbcbpgybnkvgtcb bncpiwcigbycubgcvcdnkujgfbkpbvjgkvkogbqhbejcwegtbdjgbokffngbcigubyptgbpgctn bqxgtbc

In [64]:
for sol in sols: 
    key = sol
    print("key:({})\nplaintext:\n{}\n\n".format(
            key, decode(decrypt(cipher_text, encode(key)))))

key:(opwskztkxgprqkdws)
plaintext:
hwtophhfprhxlbocuoatsxtjp o xqtfphiftorcatgodtowpdgoacftoghfcbvimoufcaogcatochwtoorcibhfxtgohwpbocfcaotbv pbsoxbouopbrtopbsodfcjtbrboxbovtfapbmopbsofrt pbsohwtftoktfbo xhtfpfmopsjtbhrftfgoacftospfxbvxpbsoprwxtjtatbhgxacftosxghxbvixgwbsoxhokpgobchoxbobbv pbsohwphohwtojcghokcbstfui ohwfbvgoktftodfcsirtaohwtftoxgobchwxbdoxboc sotbv xgwoqwphohpztgowc soccohwtoaxbsokxhwohephoapghtfui opbsxgiqsixbvodcktfokexrwoghx  oqt cbvpohcohwto mfxrp ophpbnpgocuohwtohfliqpscifgopbsoaxbktgxbvtfgohcokt geofcapbrtocfohcohetotdxrodfcgtocuoqwtoxrt pbsowxghcoxtgooohwtobcfapbxrcbeitghostvfpstaohwtotbv xgwo pbdipvtoufcaoxhgo xqtfpfmofpbzopbsoqocivwhoxbopobtko ybvipvtoucfohwtodl xhtfo xhtfphiftxxhosxsobchostghflmoxbocbtogtbgtoxqosxsobchopqgc ihb moxbhtffidhotbvixgwo xhtfphiftoqrhoxhohcczopkpmohetotbv xgwo xhtfpomoghpbspfsopbsoheftkohwtorcibhfmozprzoxbhcohwtorcbaxhxcbocuoxhp moqbucftospbhtpbopbporwmocuosxp trhgotwtbopobtko xhtfpomo pbvipvtokpgotphpq xgwtsoxbohwtxhxatocuorwpirtfoq

In [65]:
decode(secret_key) == 'cdkgznhzlvdfnzskg'

True

**:)**

---

<a rel="license" href="http://creativecommons.org/licenses/by-nc-sa/4.0/"><img alt="Creative Commons License" style="border-width:0" src="https://i.creativecommons.org/l/by-nc-sa/4.0/88x31.png" /></a><br /><span xmlns:dct="http://purl.org/dc/terms/" property="dct:title">Vigenere cryptoanalysis</span> by <a xmlns:cc="http://creativecommons.org/ns#" href="mailto:massimo.nocentini@unifi.it" property="cc:attributionName" rel="cc:attributionURL">Massimo Nocentini</a> is licensed under a <a rel="license" href="http://creativecommons.org/licenses/by-nc-sa/4.0/">Creative Commons Attribution-NonCommercial-ShareAlike 4.0 International License</a>.<br />Based on a work at <a xmlns:dct="http://purl.org/dc/terms/" href="https://github.com/massimo-nocentini/scratchpad/blob/master/chapters/crypto/vigenere-cryptoanalysis.ipynb" rel="dct:source">https://github.com/massimo-nocentini/scratchpad/blob/master/chapters/crypto/vigenere-cryptoanalysis.ipynb</a>.